In [59]:
import geopandas as gpd
import gpxpy
import gpxpy.gpx
import pandas as pd
from shapely.geometry import LineString
from shapely.geometry import Point
import fiona
from lxml import etree
from pykml.factory import KML_ElementMaker as KML
import glob
import os

In [55]:
tracks = glob.glob(r"track_data/*.gpx")

In [56]:
def mi_ign_track(gpx_file):
    # Read your GPX file
    with open(gpx_file, 'r') as gpx_file:
        gpx = gpxpy.parse(gpx_file)

    # Initialize list to store LineString objects
    lines = []

    # Extract data from GPX
    for track in gpx.tracks:
        for segment in track.segments:
            points = [(point.longitude, point.latitude) for point in segment.points]
            line = LineString(points)
            lines.append(line)

    # Create a GeoDataFrame with the LineString objects
    gdf = gpd.GeoDataFrame(geometry=lines)

    # Set the coordinate reference system (CRS) to WGS84
    gdf.set_crs(epsg=4326, inplace=True)

    return gdf

In [57]:
all_tracks = []

for track in range(len(tracks)):
  temp_track = mi_ign_track(tracks[track])
  temp_track["name"] = tracks[track].replace(".gpx", "")
  temp_track[['date', 'ID']] = temp_track['name'].str.split(' ', expand=True)
  all_tracks.append(temp_track)

In [58]:
gdftracks = gpd.GeoDataFrame(pd.concat(all_tracks, ignore_index=True))

In [62]:
if not os.path.isdir("track_results/"):
  os.makedirs("track_results/")

In [63]:
gdftracks.to_file("track_results/senderos_salidas_campo.shp")